In [ ]:
!pip install selenium

In [ ]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from concurrent.futures import ThreadPoolExecutor
from requests.exceptions import TooManyRedirects

In [ ]:
# Инициализация веб-драйвера
chrome_options = webdriver.ChromeOptions()
chrome_options.executable_path = r"Путь до chromedriver.exe"
driver = webdriver.Chrome(options=chrome_options)

# Открытие страницы
base_url = "url сайта"
news_url = "/news"
driver.get(base_url + news_url)

# Парсинг ссылок и дат
links = []
dates = []
click_counter = 0

while click_counter < 100:
    soup = BeautifulSoup(driver.page_source, "html.parser")
    articles = soup.find_all("div", class_="post_item")

    for article in articles:
        date_elem = article.find("span", class_="post_date")
        if date_elem:
            date_text = date_elem.get_text()
            dates.append(date_text)
        else:
            dates.append(np.nan)
        link_elem = article.find("a", href=True)
        link = link_elem["href"] if link_elem else None
        links.append(link)

    try:
        load_more_btn = driver.find_element(By.CLASS_NAME, "load_more_btn")
        driver.execute_script("arguments[0].click();", load_more_btn)
        WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, "post_item"))
        )
        click_counter += 1
    except Exception as e:
        print("Ошибка при нажатии на кнопку:", str(e))
        break

driver.quit()

# Создание DataFrame
data = {
    "Ссылка": links,
    "Дата": dates
}
df_Chat = pd.DataFrame(data)

# Фильтрация и сохранение датафрейма
df_Chat["Дата"] = pd.to_datetime(df_Chat["Дата"], format="%d.%m.%Y", errors="coerce")
start_date = pd.Timestamp("2023-08-31")
end_date = pd.Timestamp("2023-10-01")
filtered_df = df_Chat[(df_Chat["Дата"] >= start_date) & (df_Chat["Дата"] <= end_date)]
filtered_df.drop_duplicates(subset=['Ссылка'], inplace=True)
filtered_df.reset_index(drop=True, inplace=True)


# Обработка данных в многопоточной среде
def process_link(args):
    link, date = args
    try:
        response = requests.get(link)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, "html.parser")
        tags_elem = soup.find("div", class_="post_tags_top")
        tags_text = ", ".join(tag.get_text() for tag in tags_elem.find_all("a"))
        author_elem = soup.find("div", class_="article_meta").find("a", class_="article_author")
        author = author_elem.get_text() if author_elem else None
        title_elem = soup.find("h1")
        title = title_elem.get_text() if title_elem else None
        return {
            "Ссылка": link,
            "Название статьи": title,
            "Тэги": tags_text,
            "Автор": author,
            "Дата": date
        }
    except TooManyRedirects:
        print(f"Слишком много редиректов для ссылки: {link}")
        return None
    except Exception as e:
        print(f"Ошибка при обработке ссылки {link}: {e}")
        return None

links_and_dates = zip(filtered_df['Ссылка'], filtered_df['Дата'])
with ThreadPoolExecutor() as executor:
    results = list(executor.map(process_link, links_and_dates))

results = [result for result in results if result is not None]
df_with_additional_data = pd.DataFrame(results)
df_with_additional_data.to_csv("df_final.csv", index=False)

In [ ]:
df_with_additional_data.head()

In [ ]:
#sep.csv содержит данные из гугл аналитики
#(Страница, Просмотры страниц,	Уникальные просмотры страниц,	Средняя длительность просмотра страницы, Входы, Показатель отказов, Процент выходов, Ценность страницы)
df_read = pd.read_csv('sep.csv')

In [ ]:
columns_to_drop = ['Входы', 'Показатель отказов', 'Процент выходов', 'Ценность страницы']
df_view_no_columns = df_read.drop(columns = columns_to_drop)
df_view_no_columns.head()

In [ ]:
def add_prefix(value):
    if isinstance(value, str):
        return 'URL сайта' + value
    return value

In [ ]:
df_view_no_columns['Страница'] = df_view_no_columns['Страница'].apply(lambda x: add_prefix(x))
df_view_no_columns.head()

In [ ]:
df_with_additional_data.rename(columns={'Ссылка': 'Страница'}, inplace=True)
df_with_additional_data.head()

In [ ]:
merge_df = df_with_additional_data.merge(df_view_no_columns, on='Страница')
merge_df.head()

In [ ]:
sorted_df = merge_df.sort_values(by='Просмотры страниц', ascending=False)
sorted_df.head()

In [ ]:
merge_df.to_csv('result_table.csv', index=False)